## 采用PyTorch搭建卷积神经网络(CNN)并进行以下孤立词的简单语音识别：
#### 1. 打开后备箱
#### 2. 关闭后备箱
#### 3. 开后备箱
#### 4. 关后备箱   


## [@Vela](https://github.com/Vela-Yang) 

运行前推荐选择“代码执行程序-更改运行时类型-硬件加速器-GPU”

In [ ]:
!git clone https://github.com/Vela-Yang/Isolated-Word-Recognition-via-CNN.git Vela

In [ ]:
%matplotlib inline

导入PyTorch相关的库

In [ ]:
!pip install torchaudio
!pip install pydub
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchaudio
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
from tqdm.notebook import tqdm


代码运行的硬件环境（是否有GPU）

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# 导入数据库

In [ ]:
from Vela.MyDataset import MyDataset

建立数据集子集，包括训练集、测试集与有效集

In [ ]:
import os

class SubsetSC(MyDataset):
    def __init__(self, subset: str = None):
        super().__init__("./Vela/")

        def load_list(filename):
            filepath = os.path.join(self._path, filename)
            with open(filepath, encoding='utf8') as fileobj:
                return [os.path.join(self._path, line.strip()) for line in fileobj]

        if subset == "validation":
            self._walker = load_list("validation_list.txt")
        elif subset == "testing":
            self._walker = load_list("testing_list.txt")
        elif subset == "training":
            excludes = load_list("validation_list.txt") + load_list("testing_list.txt")
            excludes = set(excludes)
            self._walker = [w for w in self._walker if w not in excludes]


# Create training and testing split of the data. We do not use validation in this tutorial.
train_set = SubsetSC("training")
test_set = SubsetSC("testing")

waveform, sample_rate, label = train_set[0]

展示一个来自训练集的音频信息

In [ ]:
print("波形长度 {}".format(waveform.size()))
print("波形采样率 {}".format(sample_rate))

plt.plot(waveform.t().numpy())

列出音频标签

In [ ]:
labels = sorted(list(set(datapoint[2] for datapoint in train_set)))
labels

列举几个声音样本
这个是“打开后备箱”

In [ ]:
waveform_first, sample_first, label_first = train_set[0]
ipd.Audio(waveform_first.numpy(), rate=sample_first)


这个是“关后备箱”

In [ ]:
waveform_last, sample_last, label_last = train_set[-1]
ipd.Audio(waveform_last.numpy(), rate=sample_last)

对音频信号重采样（统一采样率）
由于大家设备不同，导致录制的音频也不同。
同时，8000Hz完全可以胜任语音采样。
这里听一下重采样后的信号

In [ ]:
new_sample_rate = 8000
transform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=new_sample_rate)
transformed = transform(waveform_last)
print(transformed)
plt.plot(transformed.t().numpy())
ipd.Audio(transformed.numpy(), rate=8000)

使用音频文件的索引来对应其标签

In [ ]:
def label_to_index(word):
    # 返回标签对应的数据集索引
    return torch.tensor(labels.index(word))


def index_to_label(index):
    # 返回数据集索引对应的标签
    return labels[index]


word_start = "(打)开后备箱"
index = label_to_index(word_start)
word_recovered = index_to_label(index)

print(word_start, "-->", index, "-->", word_recovered)

将信号转换成张量，适应CNN模型的输入

In [ ]:
def pad_sequence(batch):
    # 用填充0的方法让所有张量等维
    batch = [item.t() for item in batch]
    batch = torch.nn.utils.rnn.pad_sequence(batch, batch_first=True, padding_value=0.)
    return batch.permute(0, 2, 1)


def collate_fn(batch):

    # 将数据处理成几个张量的形式并合并成一个批张量( batch tensor )

    tensors, targets = [], []

    for waveform, _, label, *_ in batch:
        tensors += [waveform]
        targets += [label_to_index(label)]

    # 将几个张量转换成批张量（张量集合，也是一个张量）
    tensors = pad_sequence(tensors)
    targets = torch.stack(targets)

    return tensors, targets


batch_size = 256

if device == "cuda":
    num_workers = 1
    pin_memory = True
else:
    num_workers = 0
    pin_memory = False

train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=num_workers,
    pin_memory=pin_memory,
)
test_loader = torch.utils.data.DataLoader(
    test_set,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    collate_fn=collate_fn,
    num_workers=num_workers,
    pin_memory=pin_memory,
)

# 加载神经网络

In [ ]:
from Vela.CNNs import M11_CNN

model = M11_CNN(n_input=transformed.shape[0], n_output=len(labels))
model.to(device)
print(model)


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


n = count_parameters(model)
print("参数个数： %s" % n)

参考论文 VERY DEEP CONVOLUTIONAL NEURAL NETWORKS FOR RAW WAVEFORMS ，给出优化器的结构，其中sceduler是为了在20次迭代以后降低学习率，提高泛化能力




In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.02, weight_decay=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=40, gamma=0.1)  # 40次迭代后开始降低学习率，避免过拟合

# 训练，测试网络
--------------------------------

训练：根据数据集进行学习

测试：测试训练集的正确率



In [ ]:
def train(model, epoch, log_interval):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):

        data = data.to(device)
        target = target.to(device)

        # apply transform and model on whole batch directly on device
        data = transform(data)
        output = model(data)

        # negative log-likelihood for a tensor of size (batch x 1 x n_output)
        loss = F.nll_loss(output.squeeze(), target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 展示训练状态
        if batch_idx % log_interval == 0:
            print(f"迭代次数: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\t训练损失: {loss.item():.6f}")

        # 更新状态栏
        pbar.update(pbar_update)
        # 记录训练损失
        losses.append(loss.item())

测试函数定义

In [ ]:
def number_of_correct(pred, target):
    # 计算分类正确的选项
    return pred.squeeze().eq(target).sum().item()


def get_likely_index(tensor):
    # 找到最接近的索引
    return tensor.argmax(dim=-1)


def test(model, epoch):
    model.eval()
    correct = 0
    for data, target in test_loader:

        data = data.to(device)
        target = target.to(device)

        # 所有数据的预处理方式应该均相同
        data = transform(data)
        output = model(data)

        pred = get_likely_index(output)
        correct += number_of_correct(pred, target)

        # 更新状态栏
        pbar.update(pbar_update)
        Accuracy.append(100. * correct / len(test_loader.dataset))

    print(f"\n迭代次数 {epoch}\t准确率: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n")

开始模型训练

In [ ]:
log_interval = 20
n_epoch = 300

pbar_update = 1 / (len(train_loader) + len(test_loader))
losses = []
Accuracy = []

# 所有数据的预处理方式应该均相同
transform = transform.to(device)
with tqdm(total=n_epoch) as pbar:
    for epoch in range(1, n_epoch + 1):
        train(model, epoch, log_interval)
        test(model, epoch)
        scheduler.step()

# 展示训练损失与准确率
plt.subplot(2,1,1)
plt.plot(losses);
plt.title("训练损失");
plt.subplot(2,1,2)
plt.plot(Accuracy);
plt.title("训练后准确率");

根据输入计算其分类

In [ ]:
def predict(tensor):
    # 使用已经训练过的神经网络来预测
    tensor = tensor.to(device)
    tensor = transform(tensor)
    tensor = model(tensor.unsqueeze(0))
    tensor = get_likely_index(tensor)
    tensor = index_to_label(tensor.squeeze())
    return tensor


waveform, sample_rate, utterance, *_ = train_set[-1]

print(f"期望: {utterance}. 预测结果: {predict(waveform)}.")
ipd.Audio(waveform.numpy(), rate=sample_rate)

以下是一个分类不成功的例子
通常来说，正确率达到100%的情况不应出现

In [ ]:
for i, (waveform, sample_rate, utterance, *_) in enumerate(test_set):
    output = predict(waveform)
    if output != utterance:
        print(f"数据 #{i}. 期望: {utterance}. 预测结果: {output}.")
        ipd.Audio(waveform.numpy(), rate=sample_rate)
        break
else:
  print("所有数据均完全正确！这并不应该出现，请检查数据集是否太小或训练集与测试集重复")
  ipd.Audio(waveform.numpy(), rate=sample_rate)
ipd.Audio(waveform.numpy(), rate=sample_rate)

## 录音并尝试

In [ ]:
import sys
# js代码，录制3秒音频
def record(seconds = 3):

    from google.colab import output as colab_output
    from base64 import b64decode
    from io import BytesIO
    from pydub import AudioSegment

    RECORD = (
        b"const sleep  = time => new Promise(resolve => setTimeout(resolve, time))\n"
        b"const b2text = blob => new Promise(resolve => {\n"
        b"  const reader = new FileReader()\n"
        b"  reader.onloadend = e => resolve(e.srcElement.result)\n"
        b"  reader.readAsDataURL(blob)\n"
        b"})\n"
        b"var record = time => new Promise(async resolve => {\n"
        b"  stream = await navigator.mediaDevices.getUserMedia({ audio: true })\n"
        b"  recorder = new MediaRecorder(stream)\n"
        b"  chunks = []\n"
        b"  recorder.ondataavailable = e => chunks.push(e.data)\n"
        b"  recorder.start()\n"
        b"  await sleep(time)\n"
        b"  recorder.onstop = async ()=>{\n"
        b"    blob = new Blob(chunks)\n"
        b"    text = await b2text(blob)\n"
        b"    resolve(text)\n"
        b"  }\n"
        b"  recorder.stop()\n"
        b"})"
    )
    RECORD = RECORD.decode("ascii")

    print(f"录音开始，将持续 {seconds} 秒.")
    display(ipd.Javascript(RECORD))
    s = colab_output.eval_js("record(%d)" % (seconds * 1000))
    print("录音结束")
    b = b64decode(s.split(",")[1])

    # 录音格式定义
    fileformat = "wav"
    filename = f"_audio.{fileformat}"
    AudioSegment.from_file(BytesIO(b)).export(filename, format=fileformat)
    return torchaudio.load(filename)


waveform, sample_rate = record()
transform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=new_sample_rate)
waveform_t = transform(waveform)
transform = transform.to(device)
print(f"预测结果： {predict(waveform)}.")

ipd.Audio(waveform_t.numpy(), rate = new_sample_rate)

可以看出，搭建的神经网络准确率高，且具有不让人失望的泛化能力   
## 下面是对来自数据集外的人声进行检验，准确率也很高




In [ ]:
file = "./Vela/校验文件/" + "开后备箱1.wav"
waveform, sample_rate = torchaudio.load(file)
transform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=new_sample_rate)
waveform_t = transform(waveform)
transform = transform.to(device)
print(f"预测结果： {predict(waveform)}.")

ipd.Audio(waveform_t.numpy(), rate = new_sample_rate)

In [ ]:
#torch.save(model.state_dict(), './SavedNetwork/')